In [1]:
import numpy as np
import scipy as sp
import import_ipynb
from SV_channel import SV_channel

importing Jupyter notebook from SV_channel.ipynb
Covariance matrix:
 [[ 3.9747+0.j      1.4673+0.152j   0.497 -0.6963j -2.1676-1.8776j]
 [ 1.4673-0.152j   2.2526+0.j     -0.5101+0.2948j -0.5785-0.8164j]
 [ 0.497 +0.6963j -0.5101-0.2948j  1.1539+0.j     -0.0126-0.5614j]
 [-2.1676+1.8776j -0.5785+0.8164j -0.0126+0.5614j  2.6711+0.j    ]]

Matrix rank: 4


In [2]:
class HybridMassiveMIMO(object):
    """
    MIMO system with Hybrid Beamforming for single user case
    """
    # main system parameters
    N_s: int                              # number of data streams
    N_rf: int                             # number of RF chains
    N_tx: int                             # number of Tx antennas (UPA)
    N_rx: int                             # number of Rx antennas (UPA)
    
    # OFDM parameters
    N_ofdm: int                           # number of OFDM symbols
    N_ifft: int                           # number of points for fft
    N_c: int                              # number of sub-carriers
    N_gi: int                             # guard interval length
    mapping: dict[str, np.complex128]     # map for 1st step modulation
    
    
    def __init__(self, N_s: int = 4,
                     N_rf: int = 4, 
                     N_tx: int = 16, 
                     N_rx: int = 4,
                     N_ofdm: int = 10,
                     N_ifft: int = 512,
                     N_c: int = 450,
                     N_gi: int = 64,
                     mapping: dict[str, np.complex128] = {"00": np.complex128(1 + 0j), "01": np.complex128(0 + 1j), \
                                                         "10": np.complex128(0 - 1j), "11": np.complex128(-1 + 0j)}):
        super(HybridMassiveMIMO, self).__init__()
        self.N_s = N_s
        self.N_rf = N_rf
        self.N_tx = N_tx
        self.N_rx = N_rx
        self.N_ofdm = N_ofdm
        self.N_ifft = N_ifft
        self.N_c = N_c
        self.N_gi = N_gi
        self.mapping = mapping
        
        # compute other system parameters
        N_null = self.N_ifft - self.N_c
        
        self.n_carr = np.arange(N_null // 2, self.N_ifft - N_null / 2, dtype = int)
        self.OFDM_length = self.N_ifft + self.N_gi
        self.M = int(np.log2(len(mapping)))
    
    
    def bit_sym_mapping(self, bits: np.array) -> np.array:
        """
        Mapping the bit sequence to special modulation symbols
        """
        if len(bits) % self.M != 0:
            bits = np.hstack((bits, np.random.randint(2, size = (self.M - len(bits) % self.M))))
        
        n_bits = len(bits)
        n_syms = n_bits // self.M
        
        symbols = np.zeros(n_syms, dtype = np.complex128)
        bits = bits.reshape((-1, self.M))
        for i in range(n_syms):
            sym = ''.join([str(b) for b in bits[i]])
            symbols[i] = self.mapping[sym]
            
        return symbols
    
    
    def sym_bit_demapping(self, symbols: np.array) -> np.array:
        """
        Demapping the symbols sequence to bits
        """
        bits = np.zeros((self.M, len(symbols)), dtype = int)
        
        demapping = {v: k for k, v in self.mapping.items()}
        points = np.array(list((demapping.keys())))
        
        for i in range(len(symbols)):
            min_i = np.argmin(np.power(np.abs(points - symbols[i]), 2))
            
            bits[:, i] = np.array([int(st) for st in demapping[points[min_i]]])
        
        bits = bits.reshape((1, -1), order = 'F')[0, :]
        
        return bits
    
    
    def OFDM_mod(self, symbols: np.ndarray, n_sts: int, n_ofdm: int) -> np.ndarray:
        """
        Create OFDM modulation sequence with shape (n_sts, n_ofdm * self.OFDM_length);
        n_sts - number of streams and n_ofdm - number of OFDM symbols
        """
        OFDM = np.zeros((n_sts, self.OFDM_length, n_ofdm), dtype = np.complex128)
        
        for i in range(n_sts):
            for j in range(n_ofdm):
                OFDM[i, self.N_gi + self.n_carr, j] = symbols[i, :, j]
                OFDM[i, self.N_gi:, j] = np.fft.ifftshift(OFDM[i, self.N_gi:, j])
                OFDM[i, self.N_gi:, j] = np.fft.ifft(OFDM[i, self.N_gi:, j], norm = "ortho")
                OFDM[i, : self.N_gi, j] = OFDM[i, self.N_ifft:, j]
        
        OFDM = np.reshape(OFDM, (n_sts, -1), order = 'F')
        return OFDM
    
    
    def OFDM_demod(self, signal: np.ndarray, n_sts: int, n_ofdm: int) -> np.ndarray:
        """
        Compute OFDM demodulation; output symbols have shape (n_sts, self.N_c, n_ofdm);
        input signal has shape (n_sts, n_ofdm * self.OFDM_length)
        """
        signal = np.reshape(signal, (n_sts, self.OFDM_length, n_ofdm), order = 'F')
        signal_deofdm = np.zeros((n_sts, self.N_ifft, n_ofdm), dtype = np.complex128)
        symbols = np.zeros((n_sts, self.N_c, n_ofdm), dtype = np.complex128)
        
        for i in range(n_sts):
            for j in range(n_ofdm):
                signal_deofdm[i, :, j] = signal[i, self.N_gi:, j]
                signal_deofdm[i, :, j] = np.fft.fft(signal_deofdm[i, :, j], norm = "ortho")
                signal_deofdm[i, :, j] = np.fft.fftshift(signal_deofdm[i, :, j])
                symbols[i, :, j] = signal_deofdm[i, self.n_carr, j]
            
#         symbols = np.reshape(symbols, (n_sts, -1), order = 'F')
        return symbols

    
    def compute_Tx_signal(self, data: np.ndarray, F_rf: np.ndarray, H: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """
        Compute signal on Tx side by applying to data Hybrid Beamforming and OFDM modulation
        """
        H_eff = np.matmul(H, F_rf)
        
        F_bb = np.zeros((self.N_rf, self.N_c, self.N_s), dtype = np.complex128)
        for i in range(self.N_c):
            _, _, vh = np.linalg.svd(H_eff[:, i, :])
            v = vh.T.conj()
            F_bb[:, i, :] = v[:, : self.N_s]
            
        n_ofdm = data.shape[-1]
        F_bb_output = np.zeros((self.N_rf, self.N_c, n_ofdm), dtype = np.complex128)
        for i in range(self.N_c):
            F_bb_output[:, i, :] = np.matmul(F_bb[:, i, :], data[:, i, :])
        
        OFDM_output = self.OFDM_mod(F_bb_output, self.N_rf, n_ofdm)
        F_rf_output = np.matmul(F_rf, OFDM_output)
        
        return F_rf_output, np.mean(H_eff, axis = 1)
    
    
    def compute_channel_output(self, Tx_signal: np.ndarray, H: np.ndarray, SNR_dBHz: int) -> np.ndarray:
        """
        Compute channel's output by applying channel matrix H and adding white noise in specific bandwidth
        """
        # Channel
        Rx_signal = np.matmul(H, Tx_signal)
        
        # Compute noise power
        Rx_signal_ravel = np.ravel(Rx_signal, order = 'C')
        E_s = np.sum(np.power(np.abs(Rx_signal_ravel), 2)) / len(Rx_signal_ravel)
        SNR_dB = SNR_dBHz - 10 * np.log10(self.N_ifft / self.N_c)
        SNR = np.power(10, SNR_dB / 10)
        
        sig_noise = np.sqrt(E_s / SNR / 2)
        noise = np.random.normal(0, sig_noise, size = Rx_signal.shape) + 1j * np.random.normal(0, sig_noise, size = Rx_signal.shape)
        
        Rx_signal += noise
        
        return Rx_signal
    
    
    def compute_channel_estimation(self, H: np.ndarray, SNR_dBHz: int) -> np.ndarray:
        """
        Compute channel estimation, that is needed for applying precoding;
        return tensor with channel matrices for each sub-carrier
        """
        pilots, ltfSC = self.generate_pilots(self.N_tx)
        Tx_pilots = self.OFDM_mod(pilots, self.N_tx, self.N_tx)
        
        Rx_pilots = self.compute_channel_output(Tx_pilots, H, SNR_dBHz)
        pilots = self.OFDM_demod(Rx_pilots, self.N_rx, self.N_tx)
        
        H_est = self.channel_estimate(pilots, ltfSC, self.N_tx)
        return H_est
    
    
    def channel_estimate(self, Rx_pilots: np.ndarray, ltfSC: np.array, n_sts: int) -> np.ndarray:
        """
        Compute channel estimation by recieved pilots
        """
        P = sp.linalg.hadamard(n_sts, dtype = np.complex128).T
        ltfSC *= n_sts
        
        H_est = np.zeros((self.N_rx, self.N_c, n_sts), dtype = np.complex128)
        for i in range(self.N_rx):
            for j in range(n_sts):
                H_est[i, :, j] = np.matmul(Rx_pilots[i, :, :], P[:, j]) / ltfSC
                
        return H_est
    
    
    def equalizer_ZF(self, Rx_data: np.ndarray, H_est: np.ndarray, n_ofdm: int) -> np.ndarray:
        """
        Compute combiner W and decoding spatial-multiplexing signal
        """
        data_clean = np.zeros((self.N_s, self.N_c, n_ofdm), dtype = np.complex128)
        for i in range(self.N_c):
            W = np.matmul(np.linalg.inv(np.matmul(H_est[:, i, :].conj().T, H_est[:, i, :])), H_est[:, i, :].conj().T)
            data_clean[:, i, :] = np.matmul(W, Rx_data[:, i, :])
        
        data_clean = np.reshape(data_clean, (self.N_s, -1), order = 'F')
        return data_clean
    
    
    def generate_pilots(self, n_sts: int = 8) -> tuple[np.ndarray, np.array]:
        """
        Compute preambule for channel estimation; see MATLAB ex.: "Massive MIMO Hybrid Beamforming"
        """
        ltfSC = np.random.choice(np.array([-1, 1], dtype = np.complex128), size = self.N_c)[np.newaxis, :]
        
        P = sp.linalg.hadamard(n_sts, dtype = np.complex128)
        Nltf = n_sts
        
        pilots = np.zeros((n_sts, self.N_c, Nltf), dtype = np.complex128)
        for i in range(Nltf):
            ltf = np.matmul(P[:, i][:, np.newaxis], ltfSC)
            pilots[:, :, i] = ltf
        
        return (pilots, ltfSC)
    
    
    def compute_Q(self) -> np.ndarray:
        """
        Compute Q-matrix from QR-factorization of Fourier transform matrix F
        """
        F = np.array([[np.exp(-1j * 2 * np.pi * j * (self.n_carr[i] + 1) / self.N_ifft) for j in range(self.N_gi)] \
                      for i in range(self.N_c)])
        
        Q, _ = np.linalg.qr(F, mode = "complete")
        
        return Q
    
    
    def noise_estimate(self, Q: np.ndarray, H_est: np.ndarray) -> np.array:
        """
        Compute noise variance on each Rx antenna; return vector of variances with shape (N_rx,)
        """
        sig_noise = np.zeros(self.N_rx, dtype = np.float64)
        for i in range(self.N_rx):
            H = np.matmul(Q.conj().T, H_est[i, :, :])
            
            sig_noise[i] = np.sum(np.power(np.abs(H[self.N_gi:, :]), 2))
        
        sig_noise /= (2 * (self.N_c - self.N_gi))
        
        return sig_noise
    
    
    def compute_BER(self, F_rf: np.ndarray, H: np.ndarray, SNR_dBHz: tuple[int, int]) -> np.array:
        """
        Compute BER for SNR in SNR_dBHz interval
        """
        SNR_dBHz = np.arange(*SNR_dBHz)
        N = 10
        
        ber = np.zeros(SNR_dBHz.shape, dtype = np.float64)
        for j in range(len(SNR_dBHz)):
            for i in range(N):
                bits_tx = np.random.randint(2, size = self.M * self.N_s * self.N_c * self.N_ofdm)
                data = self.bit_sym_mapping(bits = bits_tx)
                
                data = data.reshape((self.N_s, self.N_c, self.N_ofdm), order = 'F')
                pilots, ltfSC = self.generate_pilots(self.N_s)
                data = np.concatenate((pilots, data), axis = 2)
                
                H_est = self.compute_channel_estimation(H, SNR_dBHz[j])
                
                Tx_signal, _ = self.compute_Tx_signal(data, F_rf, H_est)
                
                Rx_signal = self.compute_channel_output(Tx_signal, H, SNR_dBHz[j])
                
                Rx_data = self.OFDM_demod(Rx_signal, self.N_s, self.N_ofdm + self.N_s)
                H_est_eq = self.channel_estimate(Rx_data[:, :, : self.N_s], ltfSC, self.N_s)
                
                data_rx = self.equalizer_ZF(Rx_data[:, :, self.N_s:], H_est_eq, self.N_ofdm)
                
                bits_rx = self.sym_bit_demapping(data_rx.reshape((1, -1), order = 'F')[0, :])
                
                ber[j] += np.sum((bits_rx - bits_tx).astype(bool)) / len(bits_tx)
                
        ber /= N
                
        return ber
    
    
    def compute_C(self, F_rf: np.ndarray, H: np.ndarray, SNR_dBHz: int) -> [np.float64, np.ndarray]:
        """
        Compute channel capacity with known channel matrix and analog beamforming matrix F_rf
        """
        data_tx, _ = self.generate_pilots(self.N_s)
        pilots, ltfSC = self.generate_pilots(self.N_s)
        
        data = np.concatenate((pilots, data_tx), axis = 2)
        
        H_est = self.compute_channel_estimation(H, SNR_dBHz)
        
        Tx_signal, H_eff = self.compute_Tx_signal(data, F_rf, H_est)
        
        Rx_signal = self.compute_channel_output(Tx_signal, H, SNR_dBHz)
        
        Rx_data = self.OFDM_demod(Rx_signal, self.N_s, self.N_s + self.N_s)
        H_est_eq = self.channel_estimate(Rx_data[:, :, : self.N_s], ltfSC, self.N_s)
        
        data_rx = self.equalizer_ZF(Rx_data[:, :, self.N_s:], H_est_eq, self.N_s)
        
        data_rx = data_rx.reshape((1, -1), order = 'F')
        data_tx = data_tx.reshape((1, -1), order = 'F')
        C = np.log2(1 + np.power(np.linalg.norm(data_tx, ord = 2), 2) / np.power(np.linalg.norm(data_tx - data_rx, ord = 2), 2))
        
        return C, H_eff

In [3]:
# Example
np.random.seed(79)

qam16_gray = {"0000": np.complex128(-3 + 3j), "0001": np.complex128(-3 + 1j), "0010": np.complex128(-3 - 3j), \
              "0011": np.complex128(-3 - 1j), "0100": np.complex128(-1 + 3j), "0101": np.complex128(-1 + 1j), \
              "0110": np.complex128(-1 - 3j), "0111": np.complex128(-1 - 1j), "1000": np.complex128(3 + 3j), \
              "1001": np.complex128(3 + 1j), "1010": np.complex128(3 - 3j), "1011": np.complex128(3 - 1j), \
              "1100": np.complex128(1 + 3j), "1101": np.complex128(1 + 1j), "1110": np.complex128(1 - 3j), \
              "1111": np.complex128(1 - 1j)}

N_tx = (4, 4)
N_rx = (2, 2)
N_rf = 4
model = HybridMassiveMIMO(N_s = 4,
                     N_rf = N_rf, 
                     N_tx = N_tx[0] * N_tx[1], 
                     N_rx = N_rx[0] * N_rx[1],
                     N_ofdm = 10,
                     N_ifft = 512,
                     N_c = 450,
                     N_gi = 64,
                     mapping = qam16_gray)

F_rf = np.random.normal(0, 1, size = (N_tx[0] * N_tx[1], N_rf)) + 1j * np.random.normal(0, 1, size = (N_tx[0] * N_tx[1], N_rf))

sv_channel = SV_channel(cl = 10, 
                    rays = 2, 
                    d_phi = 7.5, 
                    d_thetta = 7.5,
                    a_r = N_rx,
                    a_t = N_tx)

H = sv_channel.compute_channel()

C, _ = model.compute_C(F_rf, H, 70)
print("Channel capacity estimation for SNR 70 dB:", C, "bps/Hz")

Channel capacity estimation for SNR 70 dB: 11.198521824187482 bps/Hz
